In [1]:
from openff.toolkit import Molecule, Topology, ForceField
#protein = Topology.from_pdb("selected_prediction_aligned.pdb").molecule(0)
#ligand = Molecule.from_file("posed_ligand.sdf")
#Topology.from_molecules([protein, ligand]).to_file("complex_topology.pdb")
#ligand = Molecule.from_smiles('[H][c]1[c]([H])[c]([Cl])[c]2[c]([c]1[H])[S][C]([C](=[O])[O-])=[C]2[Cl]')
#complex_topology = Topology.from_pdb("complex_topology.pdb", unique_molecules=[ligand])
#protein, ligand = complex_topology.molecules
lig = Molecule.from_smiles("c12c(Cl)cccc1sc(C(=O)[O-])c(Cl)2")
top = Topology.from_pdb("last_frame_lig.pdb",
                       unique_molecules=[lig])
#top = Topology.from_molecules([*top.molecules][:2])
protein = top.molecule(0)
ligand = top.molecule(1)

In [3]:
#complex_topology.visualize()

In [21]:
import rdkit
def mutate_ligand(ligand):
    ligand.partial_charges = None
    rdmol = ligand.to_rdkit()
    reactions = ["[*:1]-[*X1:2] >> [*:1][C:2]",
                 "[#6:1]-[#1:2] >> [*:1][N:2]",
                 "[#6:1]-[#1:2] >> [*:1][O:2]"
                     
                ]
    unsanitized_products = []
    for reaction in reactions:
        rxn = rdkit.Chem.rdChemReactions.ReactionFromSmarts(reaction)
        unsanitized_products.extend(rxn.RunReactants([rdmol]))
    
    products = list()
    for (product,) in unsanitized_products:
        mol_copy = rdkit.Chem.Mol(product)
        
        rdkit.Chem.SanitizeMol(mol_copy, rdkit.Chem.SANITIZE_ALL)
        rdkit.Chem.AssignStereochemistryFrom3D(mol_copy)
        rdkit.Chem.Kekulize(mol_copy, clearAromaticFlags=True)
        rdkit.Chem.SetAromaticity(mol_copy, rdkit.Chem.AromaticityModel.AROMATICITY_MDL)
        offmol = Molecule.from_rdkit(mol_copy)
        existing_metadata = [atom.metadata for atom in offmol.atoms if atom.metadata != {}][0]
        for atom in offmol.atoms:
            atom.metadata.update(existing_metadata)
            #print(atom.metadata)
        #print()
        products.append(offmol)
    return products


def display_molecule_grid(molecules, item_width=200):
    import ipywidgets as widgets
    
    items = []
    for product in molecules:
        item = widgets.Output()
        item.append_display_data(product.visualize('rdkit', width=item_width))
        items.append(item)
        
    return widgets.GridBox(
        items, 
        layout=widgets.Layout(grid_template_columns=f"repeat(3, {item_width+10}px)")
    )

In [5]:
products = mutate_ligand(ligand)
display_molecule_grid(products)

GridBox(children=(Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<IPython.core.displa…

In [19]:
import openmm
import openmm.unit as omm_unit
def relax_ligand(protein_interchange, ligand_offmol, ligand_ff, name=""):
    from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper
    from openff.interchange import Interchange
    ntkw = NAGLToolkitWrapper()
    ntkw.assign_partial_charges(ligand_offmol, "openff-gnn-am1bcc-0.1.0-rc.2.pt")
    ligand_interchange = sage.create_interchange(ligand_offmol.to_topology(), charge_from_molecules=[ligand_offmol])
    complex_interchange = protein_interchange.combine(ligand_interchange)
    omm_sys = complex_interchange.to_openmm_system()
    omm_top = complex_interchange.to_openmm_topology()
    
    
    restraint = openmm.CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2')
    omm_sys.addForce(restraint)
    restraint.addGlobalParameter('k', 4000.0 * omm_unit.kilojoules_per_mole / omm_unit.angstrom)
    restraint.addPerParticleParameter('x0')
    restraint.addPerParticleParameter('y0')
    restraint.addPerParticleParameter('z0')
    
    for atom in omm_top.atoms():
        if atom.name in ['C', 'O', 'N', 'CA', 'S1X', 'O1X', 'O2X', 'Cl1x', 'Cl2x']:
            restraint.addParticle(atom.index, complex_interchange.positions[atom.index].to_openmm())
        #elif ligand_offmol..atomatom.index 
    
    # Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
    integrator = openmm.LangevinIntegrator(
        300 * omm_unit.kelvin,
        1 / omm_unit.picosecond,
        0.002 * omm_unit.picoseconds,
    )
    
    simulation = openmm.app.Simulation(omm_top, omm_sys, integrator)
    simulation.context.setPositions(complex_interchange.positions.to_openmm())
    simulation.minimizeEnergy()
    simulation.context.setVelocitiesToTemperature(300)
    simulation.step(1000)
    from openff.units.openmm import from_openmm
    state = simulation.context.getState(getPositions=True, getForces=True, getEnergy=True)
    omm_positions = state.getPositions()
    openmm.app.PDBFile.writeFile(simulation.topology, omm_positions, open(f'{name}.pdb', 'w'))
    complex_interchange.positions = from_openmm(omm_positions)
    
    return complex_interchange, state
    

In [7]:
sage = ForceField("openff-2.2.0.offxml")


In [8]:
ff14sb = ForceField("ff14sb_off_impropers_0.0.4.offxml")
protein_interchange = ff14sb.create_interchange(protein.to_topology())
#protein_interchange = ff14sb.create_interchange(top.molecule(0).to_topology())

In [9]:
env INTERCHANGE_EXPERIMENTAL=1

env: INTERCHANGE_EXPERIMENTAL=1


In [10]:
import numpy as np
from openff.units import unit
def naive_objective(ligand):
    # CD of LEU76
    target_coord = (4.278,   1.307,   9.049)
    ligand_com = np.mean(ligand.conformers[0], axis=0).m_as(unit.angstrom)
    return np.linalg.norm(target_coord - ligand_com)


In [23]:
from openff.toolkit import Molecule, ForceField, Topology
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper
from openff.units import Quantity, unit, ensure_quantity
from openmm.app import Simulation
from openff.interchange import Interchange
from openff.interchange.drivers.openmm import get_openmm_energies

softened_ff = ForceField("openff-2.2.0.offxml")
for parameter in softened_ff.get_parameter_handler('vdW'):
    parameter.rmin_half *= 0.5
# softened_ff.parse_sources(["ff14sb_off_impropers_0.0.4.offxml"])

def single_point_energy(simulation: Simulation, positions: Quantity, box_vectors: Quantity | None) -> Quantity:
    """Calculate a single point energy with OpenMM"""
    simulation.context.setPositions(ensure_quantity(positions, 'openmm'))
    if box_vectors is not None:
        simulation.context.setPeriodicBoxVectors(*ensure_quantity(box_vectors, 'openmm'))
    state = simulation.context.getState(getEnergy=True)
    return ensure_quantity(state.getPotentialEnergy(), 'openff')

def create_integrator() -> openmm.Integrator:
    return openmm.LangevinMiddleIntegrator(
        300 * omm_unit.kelvin,
        1 / omm_unit.picosecond,
        0.002 * omm_unit.picoseconds,
    )

def interaction_objective(
    ligand: Molecule,
    previous_complex: Topology,
    protein_energy: Quantity,
    protein_interchange: Interchange,
    force_field: ForceField = softened_ff,
) -> Quantity:
    """
    Interaction energy between ligand and all but the last molecule in previous_complex.

    """
    ntkw = NAGLToolkitWrapper()
    ntkw.assign_partial_charges(ligand, "openff-gnn-am1bcc-0.1.0-rc.2.pt")

    ligand_interchange = force_field.create_interchange(ligand.to_topology(), charge_from_molecules=[ligand])
    ligand_energy = get_openmm_energies(ligand_interchange).total_energy

    complex_interchange = protein_interchange.combine(ligand_interchange)
    complex_energy = get_openmm_energies(complex_interchange).total_energy
    
    interaction_energy = complex_energy - protein_energy - ligand_energy
    return interaction_energy

In [12]:
protein_simulation = protein_interchange.to_openmm_simulation(create_integrator())

In [24]:
# %%snakeviz

import mdtraj
import numpy 

candidates = [(top.molecule(1), 999)]
protein_trajectory = mdtraj.Trajectory(protein_interchange.positions.m, mdtraj.Topology.from_openmm(protein_interchange.topology.to_openmm()))

for idx in range(15):
    print(f"round {idx}")
    candidate, energy = candidates.pop(0)
    
    print(f"Relaxing candidate")
    complex_interchange, state = relax_ligand(protein_interchange, candidate, sage, name=str(idx))
    relaxed_protein_positions = complex_interchange.topology.molecule(0).conformers[0]
    protein_interchange.positions = relaxed_protein_positions
    protein_trajectory.xyz = [*protein_trajectory.xyz, relaxed_protein_positions.m_as(unit.nanometer)]
    result_ligand = complex_interchange.topology.molecule(1)

    print(f"Computing protein point energy")
    protein_energy = single_point_energy(protein_simulation, relaxed_protein_positions, complex_interchange.box)

    print(f"generating mutations")
    mutations = mutate_ligand(result_ligand)
    print(f"pruning mutations by naive objective")

    mutations = sorted(mutations, key=lambda x:naive_objective(x))[:3]
    print(f"computing mutations")
    candidates = []
    for mutation in mutations:
        print(f"computing {mutation}")

        candidates.append((
            mutation, 
            interaction_objective(
                mutation, 
                complex_interchange.topology, 
                protein_energy,
                protein_interchange
            )
        ))
        print(f"finished with energy {candidates[-1][1]}")
    
    candidates.sort(key=lambda x: x[1])

round 0
Relaxing candidate
Computing protein point energy
generating mutations
pruning mutations by naive objective
computing mutations
computing Molecule with name '' and SMILES '[H][c]1[c]([H])[c]([C]([H])([H])[H])[c]([Cl])[c]2[c]1[S][C]([C](=[O])[O-])=[C]2[Cl]'
finished with energy -426.91946343479606 kilojoule / mole
computing Molecule with name '' and SMILES '[H][c]1[c]([H])[c]2[c]([c]([C]([H])([H])[H])[c]1[H])[C]([Cl])=[C]([C](=[O])[O-])[S]2'
finished with energy -427.11972828915543 kilojoule / mole
computing Molecule with name '' and SMILES '[H][c]1[c]([H])[c]([N]([H])[H])[c]([Cl])[c]2[c]1[S][C]([C](=[O])[O-])=[C]2[Cl]'
finished with energy -433.62663389310205 kilojoule / mole
round 1
Relaxing candidate
Computing protein point energy
generating mutations
pruning mutations by naive objective
computing mutations
computing Molecule with name '' and SMILES '[H][c]1[c]([H])[c]([N]([H])[C]([H])([H])[H])[c]([Cl])[c]2[c]1[S][C]([C](=[O])[O-])=[C]2[Cl]'
finished with energy -433.14419021

# Scratch below

In [25]:
import openmm
import openmm.unit as omm_unit
from openff.toolkit import Molecule, ForceField, Topology
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper
from openff.units import Quantity, unit, ensure_quantity
from openmm.app import Simulation
from openff.interchange import Interchange
from openff.interchange.drivers.openmm import get_openmm_energies

softened_ff = ForceField("openff-2.2.0.offxml")
for parameter in softened_ff.get_parameter_handler('vdW'):
    parameter.rmin_half *= 0.5

def relax_ligand(protein_interchange, ligand_offmol, ligand_ff, name=""):
    from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper
    from openff.interchange import Interchange
    ntkw = NAGLToolkitWrapper()
    ntkw.assign_partial_charges(ligand_offmol, "openff-gnn-am1bcc-0.1.0-rc.2.pt")
    ligand_interchange = ligand_ff.create_interchange(ligand_offmol.to_topology(), 
                                                      charge_from_molecules=[ligand_offmol])
    complex_interchange = protein_interchange.combine(ligand_interchange)
    omm_sys = complex_interchange.to_openmm_system()
    omm_top = complex_interchange.to_openmm_topology()
    
    
    restraint = openmm.CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2')
    omm_sys.addForce(restraint)
    restraint.addGlobalParameter('k', 100.0 * omm_unit.kilojoules_per_mole / omm_unit.nanometer)
    restraint.addPerParticleParameter('x0')
    restraint.addPerParticleParameter('y0')
    restraint.addPerParticleParameter('z0')
    
    for atom in omm_top.atoms():
        if atom.name in ['C', 'O', 'N', 'CA', 'S1X', 'O1X', 'O2X']:
            restraint.addParticle(atom.index, complex_interchange.positions[atom.index].to_openmm())
    
    # Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
    integrator = openmm.LangevinIntegrator(
        300 * omm_unit.kelvin,
        1 / omm_unit.picosecond,
        0.002 * omm_unit.picoseconds,
    )
    
    simulation = openmm.app.Simulation(omm_top, omm_sys, integrator)
    simulation.context.setPositions(complex_interchange.positions.to_openmm())
    simulation.minimizeEnergy()
    simulation.context.setVelocitiesToTemperature(300)
    simulation.step(1000)
    from openff.units.openmm import from_openmm
    state = simulation.context.getState(getPositions=True, getForces=True, getEnergy=True)
    omm_positions = state.getPositions()
    openmm.app.PDBFile.writeFile(simulation.topology, omm_positions, open(f'{name}.pdb', 'w'))
    complex_interchange.positions = from_openmm(omm_positions)
    
    return complex_interchange, state

def single_point_energy(simulation: Simulation, positions: Quantity, box_vectors: Quantity | None) -> Quantity:
    """Calculate a single point energy with OpenMM"""
    simulation.context.setPositions(ensure_quantity(positions, 'openmm'))
    if box_vectors is not None:
        simulation.context.setPeriodicBoxVectors(*ensure_quantity(box_vectors, 'openmm'))
    state = simulation.context.getState(getEnergy=True)
    return ensure_quantity(state.getPotentialEnergy(), 'openff')

def create_integrator() -> openmm.Integrator:
    return openmm.LangevinMiddleIntegrator(
        300 * omm_unit.kelvin,
        1 / omm_unit.picosecond,
        0.002 * omm_unit.picoseconds,
    )

def interaction_objective(
    ligand: Molecule,
    previous_complex: Topology,
    protein_energy: Quantity,
    protein_interchange: Interchange,
    force_field: ForceField = softened_ff,
) -> Quantity:
    """
    Interaction energy between ligand and all but the last molecule in previous_complex.

    """
    ntkw = NAGLToolkitWrapper()
    ntkw.assign_partial_charges(ligand, "openff-gnn-am1bcc-0.1.0-rc.2.pt")

    ligand_interchange = force_field.create_interchange(ligand.to_topology(), charge_from_molecules=[ligand])
    ligand_energy = get_openmm_energies(ligand_interchange).total_energy
    protein_interchange.positions = previous_complex.molecule(0).conformers[0]
    protein_interchange.topology.molecule(0)._conformers = [previous_complex.molecule(0).conformers[0]]
    complex_interchange = protein_interchange.combine(ligand_interchange)
    #complex_interchange = 
    complex_energy = get_openmm_energies(complex_interchange).total_energy
    print(f"{complex_energy=} {protein_energy=} {ligand_energy=}")
    interaction_energy = complex_energy - protein_energy - ligand_energy
    return interaction_energy

In [26]:
env INTERCHANGE_EXPERIMENTAL=1

env: INTERCHANGE_EXPERIMENTAL=1


In [27]:
import mdtraj
import numpy 

sage = ForceField("openff-2.2.0.offxml")

ff14sb = ForceField("ff14sb_off_impropers_0.0.4.offxml")
protein_interchange = ff14sb.create_interchange(protein.to_topology())
protein_simulation = protein_interchange.to_openmm_simulation(create_integrator())

In [29]:
candidates = [(ligand, 999, protein_interchange.positions)]
protein_trajectory = mdtraj.Trajectory(protein_interchange.positions.m, mdtraj.Topology.from_openmm(protein_interchange.topology.to_openmm()))

for idx in range(2):
    print(f"round {idx}")
    candidate, energy, protein_positions = candidates.pop(0)
    
    print(f"Relaxing candidate")
    protein_interchange.positions = protein_positions
    complex_interchange, state = relax_ligand(protein_interchange, candidate, sage, name=str(idx))
    
    relaxed_protein_positions = complex_interchange.topology.molecule(0).conformers[0]
    print(sorted(protein_interchange.positions - relaxed_protein_positions, key=lambda x:x[0])[:5])
    protein_interchange.positions = relaxed_protein_positions
    #relaxed_ligand_positions = complex_interchange.molecule(1).conformers[0]
    protein_trajectory.xyz = [*protein_trajectory.xyz, complex_interchange.positions[:-candidate.n_atoms].m_as(unit.nanometer)]
    result_ligand = complex_interchange.topology.molecule(1)
    print(result_ligand.conformers[0] - candidate.conformers[0])

    print(f"Computing protein point energy")
    protein_energy = single_point_energy(protein_simulation, 
                                         relaxed_protein_positions,
                                         #complex_interchange.topology.molecule(0).conformers[0], 
                                         complex_interchange.box)

    print(f"generating mutations")
    mutations = mutate_ligand(result_ligand)
    
    print(f"computing mutations")
    for mutation in mutations:
        print(f"computing {mutation}")
        candidates.append((
            mutation, 
            interaction_objective(
                mutation, 
                complex_interchange.topology, 
                protein_energy,
                protein_interchange
            )
        ))
        print(f"finished with energy {candidates[-1][1]}")
    
    candidates.sort(key=lambda x: x[1])

round 0
Relaxing candidate
[<Quantity([0. 0. 0.], 'nanometer')>, <Quantity([0. 0. 0.], 'nanometer')>, <Quantity([0. 0. 0.], 'nanometer')>, <Quantity([0. 0. 0.], 'nanometer')>, <Quantity([0. 0. 0.], 'nanometer')>]
[[0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0] [0.0 0.0 0.0]] angstrom
Computing protein point energy
generating mutations
computing mutations
computing Molecule with name '' and SMILES '[H][c]1[c]([Cl])[c]2[c]([c]([H])[c]1[C]([H])([H])[H])[S][C]([C](=[O])[O-])=[C]2[Cl]'
complex_energy=<Quantity(73896439.2, 'kilojoule / mole')> protein_energy=<Quantity(-19070.1082, 'kilojoule / mole')> ligand_energy=<Quantity(321.143023, 'kilojoule / mole')>
finished with energy 73915188.16474773 kilojoule / mole
computing Molecule with name '' and SMILES '[H][c]1[c]([H])[c]([C]([H])([H])[H])[c]([Cl])[c]2[c]1[S][C]([C]

KeyboardInterrupt: 

In [ ]:
nv = complex_interchange.visualize()
nv.add_representation("licorice")
nv

In [ ]:
Topology.from_molecules([protein, candidates[1][0]]).visualize()

In [ ]:
from openff.interchange.drivers.openmm import get_openmm_energies
get_openmm_energies(complex_interchange)

In [ ]:
#previous_complex = complex_interchange.topology

In [ ]:
ntkw = NAGLToolkitWrapper()
ntkw.assign_partial_charges(products[0], "openff-gnn-am1bcc-0.1.0-rc.2.pt")

ligand_interchange = sage.create_interchange(products[3].to_topology(), charge_from_molecules=[products[0]])
ligand_energy = get_openmm_energies(ligand_interchange).total_energy
protein_interchange.positions = previous_complex.molecule(0).conformers[0]
complex_interchange = protein_interchange.combine(ligand_interchange)
#complex_interchange = 
complex_energy = get_openmm_energies(complex_interchange).total_energy
print(f"{complex_energy=} {protein_energy=} {ligand_energy=}")
interaction_energy = complex_energy - protein_energy - ligand_energy


In [ ]:
nv = complex_interchange.visualize()
nv.add_representation("licorice")
nv

In [ ]:
nv = Topology.from_molecules([protein, products[3]]).visualize()
nv.add_representation("licorice")
nv

In [67]:
nv = Topology.from_molecules([protein, Molecule.from_file("posed_ligand.sdf")]).visualize()
nv.add_representation("licorice")
nv

NGLWidget()